In [116]:
%load_ext autoreload
%autoreload 2

import os
from tqdm import tqdm
from imaris_ims_file_reader.ims import ims
import dask.array as da
import pandas as pd
import numpy as np
import pickle as pkl
import napari
from cellpose import models
from skimage.io import imread,imsave
import seaborn as sns
import matplotlib.pyplot as plt

from utils import extract_regionprops, find_edge_df, mask_from_df, suppress_by_iou, compute_iou_array, add_soma_data, mark_tile_edge_objects

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Get an image

In [4]:
im_path = r'D:\data_analysis\2025_Birder_mito\88EM87C 25x25_ashlar.ome.tif'
output_dir = r'D:\data_analysis\2025_Birder_mito\C_00_analysis'

In [ ]:
store = imread(im_path, aszarr=True)
im = da.from_zarr(store,3)
im_shape = im.shape

C:\Users\KMK280\AppData\Local\Temp\ipykernel_28008\3278877819.py:1: FutureWarning: The plugin infrastructure in `skimage.io` is deprecated since version 0.25 and will be removed in 0.27 (or later). To avoid this warning, please do not pass additional keyword arguments for plugins (`**plugin_args`). Instead, use `imageio` or other I/O packages directly. See also `skimage.io.imread`.
  store = imread(im_path, aszarr=True)


In [64]:
viewer = napari.Viewer()
viewer.add_image(im)

<Image layer 'im' at 0x174dbe09e90>

## Clean SAM masks

In [34]:
properties = ['area', 'area_convex', 'area_filled', 'euler_number', 'image','bbox',
        'eccentricity', 'solidity', 'centroid', 'major_axis_length', 'minor_axis_length']


file_list = [x for x in os.listdir(output_dir) if x.startswith('tile_masks') and x.endswith('.pkl')]

df_all_list = []
for file_name in tqdm(file_list):

    print(f'Processing {file_name}')    

    df_path = os.path.join(output_dir,file_name)
    df = pd.read_pickle(df_path)
    print(f'Loaded {len(df)} masks.')

    # divide masks into separate objects
    props_list = [] 
    for ind, row in df.iterrows():
        props = extract_regionprops(row,properties,small_size=50)
        # keep origin of the data
        props['origin'] = ind
        props_list.append(props)

    props_all = pd.concat(props_list, ignore_index=True)
    props_all.columns = [ f'sc_{col}' if col == 'area' else col for col in props_all.columns]

    # drop the segmentation column
    df.drop(columns=['segmentation'], inplace=True)

    # Concatenate with the original DataFrame
    df['origin'] = df.index
    df_all = pd.merge(df, props_all,left_on = 'origin', right_on= 'origin', how = 'right').reset_index()

    df_all['label'] = df_all.index + 1

    print(f'Kept {len(df_all)} axons.')

    df_all_list.append(df_all)

  0%|          | 0/20 [00:00<?, ?it/s]

Processing tile_masks_00000_00000.pkl
Loaded 231 masks.


  5%|▌         | 1/20 [00:01<00:34,  1.79s/it]

Kept 243 axons.
Processing tile_masks_00000_00724.pkl
Loaded 396 masks.


 10%|█         | 2/20 [00:04<00:42,  2.34s/it]

...

Kept 477 axons.
Processing tile_masks_01737_02172.pkl
Loaded 404 masks.


 95%|█████████▌| 19/20 [00:49<00:02,  2.95s/it]

Kept 430 axons.
Processing tile_masks_01737_02635.pkl
Loaded 343 masks.


100%|██████████| 20/20 [00:52<00:00,  2.62s/it]

Kept 361 axons.


## Suppress overlapping objects

In [96]:
df = pd.concat(df_all_list, ignore_index=True)
df.label = df.index + 1
print(f'Starting number of axons: {len(df)}')

# move into the full image coordinate system
df['bbox-0'] = df['bbox-0'] + df['tile_row_start']
df['bbox-1'] = df['bbox-1'] + df['tile_col_start']
df['bbox-2'] = df['bbox-2'] + df['tile_row_start']
df['bbox-3'] = df['bbox-3'] + df['tile_col_start']
df['centroid-0'] = df['centroid-0'] + df['tile_row_start']
df['centroid-1'] = df['centroid-1'] + df['tile_col_start']

# remove not promissing objects based on updated parameters
df = df.loc[((df['euler_number'] < 1) & (df.sc_area/df.area_filled < 0.9)),:]

# remove edge objects of internal tile boundaries
df = mark_tile_edge_objects(df, pad=5, filter='not_edge')
print(f'Number of axons after removing edge objects: {len(df)}')
df['edge_ring'] = False

# supress by iou
df = df.reset_index(drop=True)
df_res = suppress_by_iou(df, iou_threshold=0.2)
df_res = df_res.loc[df_res.keep == 1,:]
print(f'Number of axons after suppressing overlaping objects: {len(df_res)}')

Starting number of axons: 7533
Number of axons after removing edge objects: 2022
Number of axons after suppressing overlaping objects: 548


## Find axons soma

In [118]:
inside_props = ['area', 'eccentricity', 'major_axis_length', 'minor_axis_length', 'centroid']
pad = 3

df_final = add_soma_data(df_res, inside_props, pad = pad)

# clean up labels
df_final = df_final.reset_index(drop=True)
df_final['label'] = df_final.index + 1
df_final['inside_label'] = df_final.index + 1

# add images of axons


## Visualize cleaned masks

In [110]:
# Visualize myelin rings

mask = mask_from_df(df_final,im.shape, prefix='')
viewer.add_labels(mask, name='myelin rings', opacity = 0.8)

# Visualize detected axons

mask = mask_from_df(df_final,im.shape, prefix='inside_')
viewer.add_labels(mask, name='axons', opacity = 0.4)

<Labels layer 'axons' at 0x1758a19eed0>

## Save data frame

In [111]:
df_final.to_pickle('I:\CBI\Jonathan\CLEM\Birder\88EM87C_sample_masks_center.pkl')